In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder




In [56]:
data_main = pd.read_csv("bengaluru_house_prices.csv")


In [57]:
data_main=data_main.drop(["society","area_type","availability","balcony"], axis = 1)

In [58]:
data_main["location"]=data_main["location"].fillna("Sarjapur  Road")
data_main["size"]=data_main["size"].fillna("2 BHK")
data_main["bath"]=data_main["bath"].fillna(data_main["bath"].median())



In [59]:
data_main.sample(15)

,location,size,total_sqft,bath,price
13065,Whitefield,3 BHK,1564,3.0,103.0
4658,4th Block Koramangala,3 BHK,1800,3.0,140.0
9017,Shanthala Nagar,3 BHK,1600,3.0,180.0
415,Amco Colony,3 BHK,1760,3.0,88.0
10050,Old Madras Road,5 BHK,4500 - 5540,7.0,287.0
9784,KSRTC Layout,3 BHK,1215,2.0,50.0
6686,High grounds,3 BHK,1900,2.0,190.0
8581,Margondanahalli,2 Bedroom,900,2.0,65.0
10813,Kasavanhalli,3 BHK,1819,3.0,150.0
6592,Langford Town,4 BHK,697Sq. Meter,4.0,211.0


In [60]:
data_main["bhk"]=data_main["size"].str.split().str.get(0).astype(int)

In [61]:
data_main.sample(12)

,location,size,total_sqft,bath,price,bhk
682,Marathahalli,2 BHK,1019,2.0,49.85,2
6931,Rajaji Nagar,3 BHK,1615,3.0,175.00,3
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.49,2
7165,Sarjapur,3 BHK,1364,2.0,56.00,3
1363,Thanisandra,1 RK,510,1.0,25.25,1
13269,Electronic City,3 Bedroom,2010,3.0,201.00,3
11877,Sarjapur Road,4 BHK,3335,4.0,300.00,4
7697,Jakkur,3 BHK,1858,3.0,123.00,3
8531,Varthur Road,2 BHK,1033,2.0,32.00,2
11791,GM Palaya,1 BHK,675,1.0,35.00,1


In [62]:
data_main=data_main.drop("size",axis=1)

In [63]:
data_main

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,Richards Town,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [64]:
data_main["location"].value_counts()

location
Whitefield                         540
Sarjapur  Road                     400
Electronic City                    302
Kanakpura Road                     273
Thanisandra                        234
                                  ... 
3rd Stage Raja Rajeshwari Nagar      1
Chuchangatta Colony                  1
Electronic City Phase 1,             1
Chikbasavanapura                     1
Abshot Layout                        1
Name: count, Length: 1305, dtype: int64

In [65]:
data_main["location"]=data_main["location"].apply(lambda x:x.strip())

In [66]:
location_count = data_main["location"].value_counts()
location_count

location
Whitefield                              541
Sarjapur  Road                          400
Electronic City                         304
Kanakpura Road                          273
Thanisandra                             237
                                       ... 
Duddanahalli                              1
Doddanakunte                              1
Jogupalya                                 1
Subhash Nagar                             1
Kengeri Satellite Town KHB Apartment      1
Name: count, Length: 1294, dtype: int64

In [67]:
location_10 = location_count[location_count<=11]
location_10

location
HAL 2nd Stage                           11
Kodigehalli                             11
Banjara Layout                          11
Narayanapura                            11
Thyagaraja Nagar                        11
                                        ..
Duddanahalli                             1
Doddanakunte                             1
Jogupalya                                1
Subhash Nagar                            1
Kengeri Satellite Town KHB Apartment     1
Name: count, Length: 1064, dtype: int64

In [68]:
data_main["location"]= data_main["location"].apply(lambda x: "others" if x in location_10 else x)
data_main["location"].sample(15)


4180               others
10018    Chikka Tirupathi
6331          Thanisandra
6982         Rajaji Nagar
13050         Thanisandra
2887         Haralur Road
7452           Whitefield
2108             Kothanur
8123            Gubbalala
5442               others
3671          Bhoganhalli
5858             Shampura
11128              Jigani
709                others
7630            Yelahanka
Name: location, dtype: object

In [70]:
data_main

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,others,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [71]:
data_main.shape

(13320, 5)

In [ ]:
# locations_reshaped = data_main['location'].values.reshape(-1, 1)
# onehot = OneHotEncoder(sparse_output=False)  
# location_encoded = onehot.fit_transform(locations_reshaped)
# location_encoded_df = pd.DataFrame(location_encoded, columns=onehot.get_feature_names_out(['location']))
# data_main = pd.concat([data_main, location_encoded_df], axis=1)
# data_main = data_main.drop('location', axis=1)


In [72]:
data_main.sample(15)

,location,total_sqft,bath,price,bhk
8810,others,630,2.0,70.00,1
13223,others,1155,2.0,64.00,2
1476,HSR Layout,1372,2.0,61.00,2
6627,others,1020,2.0,60.00,2
5057,Uttarahalli,1320,2.0,55.44,3
9859,Gollarapalya Hosahalli,1905,3.0,86.00,4
9129,Mysore Road,1410 - 1710,2.0,83.30,3
11736,Kogilu,2456,7.0,85.00,7
9621,Sarjapura - Attibele Road,3750,6.0,295.00,5
9598,Rajaji Nagar,6500,5.0,1400.00,4


In [73]:
import re

def find_unit(x):
    x = str(x).strip()
    if re.match(r'^\d+(\.\d+)?$', x):
        return 'Number'
    elif '-' in x:  
        return 'Range'
    else:
        unit = re.sub(r'[\d\.\-\s]', '', x)
        return unit if unit else 'Unknown'
unit_type = data_main['total_sqft'].apply(find_unit)
print(unit_type.unique())


['Number' 'Range' 'SqMeter' 'Perch' 'SqYards' 'Acres' 'Cents' 'Guntha'
 'Grounds']


In [74]:
data_main.sample(10)

,location,total_sqft,bath,price,bhk
9085,Bharathi Nagar,1664,3.0,69.89,3
12894,Ramamurthy Nagar,960,2.0,35.00,2
11712,Ramamurthy Nagar,1208,2.0,43.50,3
9549,Kanakpura Road,1100,3.0,58.00,3
3953,Electronic City,1500,2.0,100.00,3
8812,Domlur,1740,3.0,120.00,3
584,others,1578,3.0,180.00,3
6265,others,1900,3.0,155.00,3
9608,Vidyaranyapura,1050,2.0,50.00,2
1727,others,1700,2.0,550.00,4


In [75]:

import re

def convert_sqmt_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'sq. meter' in x_lower or 'sqmeter' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 10.7639
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x




In [76]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_sqmt_to_sqft)

In [77]:
data_main[3225:]

,location,total_sqft,bath,price,bhk
3225,others,1069,2.0,55.0,2
3226,Sarjapura - Attibele Road,829,2.0,22.8,2
3227,Koramangala,1900,2.0,200.0,2
3228,others,32722.26,5.0,140.0,5
3229,Sarjapur Road,1300,3.0,60.0,3
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.0,5
13316,others,3600,5.0,400.0,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.0,2
13318,Padmanabhanagar,4689,4.0,488.0,4


In [78]:
def convert_perch_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'perch' in x_lower or '.perch' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 272.25
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x


In [79]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_perch_to_sqft)

In [80]:
data_main.sample(15)

,location,total_sqft,bath,price,bhk
10091,others,1250,3.0,40.00,3
3533,Balagere,1205.47,2.0,81.00,2
9020,others,1140,2.0,49.00,2
5422,Sarakki Nagar,1200,3.0,128.00,3
2170,Electronic City,870,2.0,39.50,2
5929,Vijayanagar,1484,3.0,79.19,3
4185,8th Phase JP Nagar,1080,2.0,38.00,2
7150,Bellandur,1200,2.0,52.00,2
8753,Akshaya Nagar,1820,4.0,95.00,3
2519,Kodichikkanahalli,2700,7.0,125.00,5


In [81]:
def convert_yards_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'sq. yards' in x_lower or 'sq.yards' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 9
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x


In [82]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_yards_to_sqft)

In [83]:
data_main[1820:]

,location,total_sqft,bath,price,bhk
1820,2nd Stage Nagarbhavi,1200,4.0,285.00,5
1821,Sarjapur,14166.0,3.0,76.00,3
1822,Nagarbhavi,884,2.0,36.00,1
1823,Haralur Road,1194,2.0,46.00,2
1824,Hoodi,1026,2.0,60.87,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,others,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [84]:
def convert_acres_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'acres' in x_lower or '.acres' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 43560
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [85]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_acres_to_sqft)

In [86]:
data_main[7726:]

,location,total_sqft,bath,price,bhk
7726,Kanakpura Road,54885.6,1.0,125.0,1
7727,others,5422,6.0,1900.0,4
7728,Hennur Road,1020,2.0,47.0,2
7729,others,1000,2.0,150.0,2
7730,others,800,3.0,59.5,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.0,5
13316,others,3600,5.0,400.0,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.0,2
13318,Padmanabhanagar,4689,4.0,488.0,4


In [87]:
def convert_cents_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'cents' in x_lower or '.cents' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 435.6
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [88]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_cents_to_sqft)

In [89]:
def convert_guntha_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'guntha' in x_lower or '.guntha' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 1089
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [90]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_guntha_to_sqft)

In [91]:
def convert_grounds_to_sqft(x):
    if not isinstance(x, str):
        return x 
    x_lower = x.lower()
    if 'ground' in x_lower or '.grounds' in x_lower:
        try:
            match = re.search(r'(\d+\.?\d*)', x)            
            if match:
                num = float(match.group(1))
                sqft = num * 2400
                return round(sqft, 2)
            else:
                return x
        except Exception:
            return x
    else:
        return x

In [92]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_grounds_to_sqft)

In [93]:
data_main["total_sqft"].nunique

<bound method IndexOpsMixin.nunique of 0        1056
1        2600
2        1440
3        1521
4        1200
         ... 
13315    3453
13316    3600
13317    1141
13318    4689
13319     550
Name: total_sqft, Length: 13320, dtype: object>

In [94]:
def find_unit(x):
    x = str(x).strip()
    if re.match(r'^\d+(\.\d+)?$', x):
        return 'Number'
    elif '-' in x:  
        return 'Range'
    else:
        unit = re.sub(r'[\d\.\-\s]', '', x)
        return unit if unit else 'Unknown'
unit_type = data_main['total_sqft'].apply(find_unit)
print(unit_type.unique())

['Number' 'Range']


In [95]:
def convert_unit(x):
    x=str(x)
    a = x.split("-")
    if len(a) == 2:  # case: "200-300"
        return (float(a[0]) + float(a[1])) / 2
    try:  # case: "250"
        return float(a[0])
    except:  # case: "abc"
        return None


In [96]:
data_main["total_sqft"]=data_main["total_sqft"].apply(convert_unit)

In [97]:
data_main[4894:]

,location,total_sqft,bath,price,bhk
4894,Bommanahalli,1625.0,2.0,87.72,3
4895,Whitefield,1400.0,2.0,80.00,2
4896,Electronic City,1310.0,2.0,37.83,3
4897,others,1100.0,2.0,48.00,2
4898,others,1230.0,2.0,53.51,3
...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5
13316,others,3600.0,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2
13318,Padmanabhanagar,4689.0,4.0,488.00,4


In [98]:
data_main.describe()

,total_sqft,bath,price,bhk
count,1.332000e+04,13320.000000,13320.000000,13320.000000
mean,1.911597e+03,2.688814,112.565627,2.802778
std,1.727690e+04,1.338754,148.971674,1.294496
min,1.000000e+00,1.000000,8.000000,1.000000
25%,1.100000e+03,2.000000,50.000000,2.000000
50%,1.277250e+03,2.000000,72.000000,3.000000
75%,1.680000e+03,3.000000,120.000000,3.000000
max,1.306800e+06,40.000000,3600.000000,43.000000


In [99]:
data_main.shape

(13320, 5)

In [100]:
data_main = data_main[((data_main["total_sqft"]/data_main["bhk"]>=300))]

In [101]:
data_main.shape

(12572, 5)

In [102]:
data_main.describe()

,total_sqft,bath,price,bhk
count,1.257200e+04,12572.000000,12572.000000,12572.000000
mean,1.967604e+03,2.560770,111.483919,2.651766
std,1.778157e+04,1.081684,151.965887,0.981470
min,3.000000e+02,1.000000,8.440000,1.000000
25%,1.116000e+03,2.000000,49.000000,2.000000
50%,1.300000e+03,2.000000,70.000000,3.000000
75%,1.703250e+03,3.000000,115.000000,3.000000
max,1.306800e+06,16.000000,3600.000000,16.000000


In [103]:
data_main['price_per_sqft'] = data_main['price'] * 100000 / data_main['total_sqft']

C:\Users\ROHIN\AppData\Local\Temp\ipykernel_11300\4172819595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_main['price_per_sqft'] = data_main['price'] * 100000 / data_main['total_sqft']


In [104]:
data_main

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000
...,...,...,...,...,...,...
13315,Whitefield,3453.0,4.0,231.00,5,6689.834926
13316,others,3600.0,5.0,400.00,4,11111.111111
13317,Raja Rajeshwari Nagar,1141.0,2.0,60.00,2,5258.545136
13318,Padmanabhanagar,4689.0,4.0,488.00,4,10407.336319


In [105]:
data_main["price_per_sqft"].describe()

count     12572.000000
mean       6294.446252
std        4163.245711
min           2.257423
25%        4203.059701
50%        5291.005291
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64

In [106]:


def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        
        gen_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_output = pd.concat([df_output, gen_df], ignore_index=True)
    return df_output

In [107]:
data_main = remove_outliers_sqft(data_main)

In [109]:
data_main.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,10347.000000,10347.000000,10347.000000,10347.000000,10347.000000
mean,1576.866752,2.473374,91.275263,2.575626,5652.313280
std,6478.270547,0.982026,86.188437,0.900070,2241.980923
min,300.000000,1.000000,10.000000,1.000000,33.210897
25%,1109.000000,2.000000,49.000000,2.000000,4241.109085
50%,1286.000000,2.000000,67.000000,2.000000,5175.202156
75%,1650.000000,3.000000,100.000000,3.000000,6433.765573
max,653400.000000,16.000000,2200.000000,16.000000,24000.000000
